In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')
from src.cell_embedding import CellEmbedding
from src.toolkit import get_vec_tarr, get_sheet_names, get_sheet_tarr
from src.cell_classification import CellLSTMClassification

In [ ]:
get_sheet_names('../sample_data/test.xls', 'xls')

In [ ]:
tarr = get_sheet_tarr('../sample_data/test.xls', '17robtbl3', 'xls')

In [ ]:
import pickle
import numpy as np

sent_enc = np.load('../sample_data/sentence_encodings.npy', mmap_mode='r')
s2i = dict((s,i) for i, s in enumerate(pickle.load(open('../sample_data/sentences.pickle', 'rb'))))

border_cases = ['__RIGHT__', '__LEFT__', '__TOP__', '__BOTTOM__', '__NULL__']

S = sent_enc.shape[0]

border_case_encs = np.zeros([5,4096], dtype=float)
border_case_encs[0,:] = 1 ; s2i[border_cases[0]] = S
border_case_encs[1,:] = 1 ; s2i[border_cases[1]] = S+1
border_case_encs[2,:] = 1 ; s2i[border_cases[2]] = S+2
border_case_encs[3,:] = 1 ; s2i[border_cases[3]] = S+3
border_case_encs[4,:] = 0 ; s2i[border_cases[4]] = S+4

class sent_enc_wrapper:
    def __init__(self, sent_enc, special_enc):
        self.n , self.m = sent_enc.shape
        self.sent_enc = sent_enc
        self.special_enc = special_enc
        self.shape = (self.n+5, self.m)
        pass
    
    def __getitem__(self, ind):
        if ind >= self.n:
            return self.special_enc[ind - self.n]
        else:
            return self.sent_enc[ind]
        

#sent_enc = np.vstack([sent_enc, border_case_encs])
sent_enc = sent_enc_wrapper(sent_enc, border_case_encs)

In [ ]:
sys.path.append('../src/') ## the pickled model needs the 'cell_embedding.py' file in path
ce = CellEmbedding(4096, 100, 8, bn=False, cuda=False)
ce.load_model('../sample_data/test_model.pickle', sent_enc, s2i)

In [ ]:
vtarr = get_vec_tarr(tarr, ce)

In [ ]:
cl = pickle.load(open('../sample_data/test_cl_model.pickle', 'rb'))

In [ ]:
vtarr.shape

In [ ]:
pred, lp = cl.predict([vtarr])

In [ ]:
labels = ['header', 'data', 'metadata', 'derived', 'attribute', 'null', 'notes']
vfunc = np.vectorize(lambda x: labels[x])
vfunc(pred)